In [1]:
!pip install  fasttext numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp311-cp311-linux_x86_64.whl size=4313501 sha256=4b37a1908af6b67e24c4235d021be8e076b7e00bb5be50115c7072559a552e87
  Stored in directory: /root/.cache/pip/wheels/65/4f/35/5057db0249224e9ab55a513fa6b79451473ceb7713017823c3
Successfully built fasttext


In [2]:
!pip install fasttext numpy==1.26.4 pandas==2.0.0 --ignore-installed

  Using cached fasttext-0.9.3-cp311-cp311-linux_x86_64.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.1 MB/s eta 0:00:00
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
  Using cached setuptools-80.4.0-py3-none-any.whl.metadata (6.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 101.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 90.9 MB/s eta 0:00:00
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 35.1 MB/s eta 0:00:00
Using cached setuptools-80.4.0-py3-none-any.whl (1.2 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 25.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, wh

In [1]:
import fasttext
import numpy as np

In [2]:
import pandas as pd
from google.colab import files

In [4]:
pd.__version__

'2.0.0'

In [5]:
def prepare_data(file_path):
    df = pd.read_csv(file_path)
    df["text"] = "__label__" + df["LFFUNC"] + "_" + df["LFVAL"] + " " + df["LFARG"]
    df["text"].to_csv("fasttext_train.txt", index=False, header=False)

    return df

In [6]:
def train_model():
    model = fasttext.train_supervised(
        input="fasttext_train.txt",
        lr=0.1,
        epoch=100,
        wordNgrams=2,
        dim=100,
        verbose=2
    )
    return model

In [7]:
def predict_lf(model, function, word, top_k=5):
    labels, probs = model.predict(word, k=1000)

    results = []
    for label, prob in zip(labels, probs):
        if f"__label__{function}_" in label:
            lf_val = label.replace(f"__label__{function}_", "")
            results.append((lf_val, prob))

    return sorted(results, key=lambda x: -x[1])[:top_k]

In [8]:
if __name__ == "__main__":
    uploaded = files.upload()
    file_name = next(iter(uploaded))

    df = prepare_data(file_name)
    model = train_model()

    test_cases_old = [
        ('MAGN', 'довод'),
        ('OPER1', 'домино'),
        ('OPER2', 'арест'),
        ('INCEPOPER1', 'азарт'),
        ('FUNC0', 'дорога'),
        ('INCEPFUNC0', 'день'),
        ('CAUSFUNC0', 'встреча'),
        ('REAL1', 'газета'),
        ('REAL1-M', 'долг')
    ]

    test_cases_new = [
        ('LOC', 'дом'),
        ('OPER1', 'оценка'),
        ('MAGN', 'друг'),
        ('ADV2-UN', 'причина'),
        ('ADV1-UN', 'надежда'),
        ('CAUSFUNC0', 'заседание'),
        ('FUNC0', 'дело'),
        ('INCEPOPER1', 'право'),
        ('OPER2', 'внимание')
    ]

    for func, arg in test_cases_old:
        print(f"\nТоп-5 вариантов для {func}({arg}):")
        predictions = predict_lf(model, func, arg)

        if not predictions:
            print("Нет предсказаний для этой комбинации")
        else:
            for i, (val, prob) in enumerate(predictions, 1):
                print(f"{i}. {val} (вероятность: {prob:.4f})")


    for func, arg in test_cases_new:
        print(f"\nТоп-5 вариантов для {func}({arg}):")
        predictions = predict_lf(model, func, arg)

        if not predictions:
            print("Нет предсказаний для этой комбинации")
        else:
            for i, (val, prob) in enumerate(predictions, 1):
                print(f"{i}. {val} (вероятность: {prob:.4f})")

Saving lf_data_words.csv to lf_data_words.csv

Топ-5 вариантов для MAGN(довод):
1. убедительный (вероятность: 0.7840)
2. железный (вероятность: 0.0014)
3. яркий (вероятность: 0.0010)
4. убедительно (вероятность: 0.0003)
5. безнадежно (вероятность: 0.0003)

Топ-5 вариантов для OPER1(домино):
1. играть (вероятность: 0.9246)
2. сыграть (вероятность: 0.0087)
3. проводить (вероятность: 0.0034)
4. вести (вероятность: 0.0029)
5. входить (вероятность: 0.0003)

Топ-5 вариантов для OPER2(арест):
1. сидеть (вероятность: 0.0871)
2. подвергать (вероятность: 0.0019)
3. подвергаться (вероятность: 0.0014)
4. переносить (вероятность: 0.0004)
5. носить (вероятность: 0.0002)

Топ-5 вариантов для INCEPOPER1(азарт):
1. входить (вероятность: 0.3758)
2. вступать (вероятность: 0.0034)
3. заключать (вероятность: 0.0034)
4. объявлять (вероятность: 0.0004)
5. обретать (вероятность: 0.0003)

Топ-5 вариантов для FUNC0(дорога):
1. проходить (вероятность: 0.2428)
2. пролегать (вероятность: 0.0094)
3. раздаваться (ве